In [16]:
import pandas as pd
# cancer = "HNSC"
# cancer = "COAD"
cancer = "STAD"
d = pd.read_csv(f"Data/{cancer}/gene_biome_{cancer}.csv", index_col=0)

In [20]:
# Visualize data with PCA, making sure it's standardized
y = d["tumor"]
x = d.drop(["tumor"], axis=1)
d_ge = x.drop(x.columns[range(0,11)], axis=1)
print(d_ge.columns)

Index(['RPS4Y1', 'XIST', 'KRT5', 'AGR2', 'CEACAM5', 'KRT6A', 'KRT14',
       'CEACAM6', 'DDX3Y', 'KDM5D',
       ...
       'FAM129A', 'C8orf48', 'CDK5R1', 'FAM81A', 'C13orf18', 'GDPD3', 'SMAGP',
       'C2orf85', 'POU5F1B', 'CHST2'],
      dtype='object', length=5000)


In [21]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pcs = pca.fit_transform(d_ge)
pc_df = pd.DataFrame(data = pcs, columns = ['principal component 1', 'principal component 2'])

In [23]:
# final_df = pd.concat([pc_df, ])
import matplotlib as plt
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['tumor']
colors = ['r', 'g']
plt.scatter(pc_df, y, c=colors, cmap=plt.colors.ListedColormap(colors))

# for target, color in zip(targets,colors):
#     indicesToKeep = pc_df['tumor'] == 1
#     ax.scatter(pc_df.loc[indicesToKeep, 'principal component 1']
#                , pc_df.loc[indicesToKeep, 'principal component 2']
#                , c = color
            #    , s = 50)
ax.legend(targets)
ax.grid()

AttributeError: module 'matplotlib' has no attribute 'figure'